# Dry & Wet Growing Media Model Evaluation
This notebook takes the samples from box 40 and evalautes all models' performance on the 2022-08-15 and 2022-08-24 dates. Metrics include DICE, +IOU, AP, and inference time (in seconds).

In [1]:
import time
import sys
import os
import matplotlib.pyplot as plt

os.environ["CUDA_HOME"] = "/apps/compilers/cuda/12.2.2"

import spectral

import torch
from torch.utils.data import DataLoader

from src.PLTrainer import validate_net, test_net
from src.Experiments.params_HyperPRI import ExpRedGreenBluePRI, ExpHyperspectralPRI

[2024-06-18 07:55:59,430] [INFO] [real_accelerator.py:203:get_accelerator] Setting ds_accelerator to cuda (auto detect)


/lustre/blue/azare/chang.spencer/textures/compiler_compat/ld: cannot find -laio: No such file or directory
collect2: error: ld returned 1 exit status


 [WARNING]  async_io requires the dev libaio .so object and headers but these were not found.
 [WARNING]  async_io: please install the libaio-devel package with yum
 [WARNING]  If libaio is already installed (perhaps from source), try setting the CFLAGS and LDFLAGS environment variables to where it can be found.
 [WARNING]  Please specify the CUTLASS repo directory as environment variable $CUTLASS_PATH
 [WARNING]  sparse_attn requires a torch version >= 1.5 and < 2.0 but detected 2.0
 [WARNING]  using untested triton version (2.0.0), only 1.0.0 is known to be compatible


## Model Parameter Setup

In [2]:
start_split = 0
num_splits = 5   # Assuming multiple splits
rel_call_path = os.path.dirname(os.path.abspath("./test_models.ipynb"))

update_params = {
    'model_name': [
         'UNET',
         'SpectralUNET',
         'CubeNET',
    ],
    'dataset': [
        'RGB',
        'HSI',
        'HSI',
    ],
    # 'device': [
    #     'cpu',
    #     'cpu',
    #     'cpu',
    # ],
    'spectral_bn_size': [
        0,
        1650,
        0,
    ],
    'cube_featmaps': [
        0,
        0,
        64,
    ],
    'criterion': [
        torch.nn.BCEWithLogitsLoss(),
        torch.nn.BCEWithLogitsLoss(),
        torch.nn.BCEWithLogitsLoss(),
    ]
}
thresholds = [
    [0.36, 0.41, 0.42, 0.56, 0.38],   # UNET
    [0.45, 0.39, 0.48, 0.36, 0.28],   # SpectralUNET
    [0.33, 0.46, 0.39, 0.46, 0.27],   # CubeNET
]

segmaps = [
    False,
    False,
    False,
]

models = update_params['model_name']
datasets = update_params['dataset']

## Get Initial Test Metrics

In [3]:
for run in range(start_split, num_splits):
    print(f" ********** Split {run+1} **********")

    for m_idx, (m, dset) in enumerate(zip(models, datasets)):
        change_params = {}
        for k_idx, k in enumerate(update_params):
            change_params[k] = update_params[k][m_idx]

        split_no = run + 1
        if dset.lower() == 'rgb':
            exp_params = ExpRedGreenBluePRI(rel_call_path, split_no=run+1, augment=False)
            # Switch to a different model (ie. change internal parameter strings)
            exp_params.change_network_param(m, rel_call_path, run+1, model_params=change_params)  # Num bins
        else:
            exp_params = ExpHyperspectralPRI(rel_call_path, split_no=run+1)
            # Switch to a different model (ie. change internal parameter strings)
            exp_params.change_network_param(m, rel_call_path, run+1, model_params=None)  # Num bins
        exp_params.json_dir['test'] = os.path.join(f"{exp_params.data_dir}", "data_splits", "test.json")

        print(f"   Model: {exp_params.model_param_str}")
        print(f"   Test JSON: {exp_params.json_dir['test']}")
        # pr_curve_info = validate_net(exp_params.get_val_data(),
        test_net(exp_params.get_test_data(), exp_params,
                 best_threshold=thresholds[m_idx][run],
                 save_segmaps=segmaps[m_idx])


 ********** Split 1 **********
   Model: UNET
   Test JSON: /blue/azare/chang.spencer/HyperPRI/Datasets/HyperPRI/data_splits/test.json
   LOADING FROM CKPT FILE: /blue/azare/chang.spencer/HyperPRI/Saved_Models/RGB/UNET/Run_1/Checkpoints/epoch=69-val_loss=0.079-val_dice=0.816.ckpt


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
SLURM auto-requeueing enabled. Setting signal handlers.
/lustre/blue/azare/chang.spencer/textures/lib/python3.10/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:438: PossibleUserWarning: The dataloader, predict_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 128 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


Predicting DataLoader 0: 100%|██████████| 2/2 [00:30<00:00, 15.24s/it]
Threshold 0.360:
      Pixel Acc: 0.762
      DICE     : 0.163
      +IOU     : 0.089
      Avg Prec : 0.208

      Conf Mat : [0.7587399482727051, 0.2412600815296173]
                 [0.10382945090532303, 0.8961705565452576]
   Model: SpectralUNET_1650
   Test JSON: /blue/azare/chang.spencer/HyperPRI/Datasets/HyperPRI/data_splits/test.json
   LOADING FROM CKPT FILE: /blue/azare/chang.spencer/HyperPRI/Saved_Models/HSI/SpectralUNET_1650/Run_1/Checkpoints/epoch=341-val_loss=0.166-val_dice=0.669.ckpt
      (Converting DeepSpeed ZeRO-2 Checkpoint...)
Processing zero checkpoint './checkpoint'
Detected checkpoint of type zero stage 2, world_size: 2
Parsing checkpoint created by deepspeed==0.14.2
Reconstructed fp32 state dict with 38 params 30388051 elements
Saving fp32 state dict to deepspeed.ckpt


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
SLURM auto-requeueing enabled. Setting signal handlers.
/lustre/blue/azare/chang.spencer/textures/lib/python3.10/site-packages/spectral/io/envi.py:175: UserWarning: Parameters with non-lowercase names encountered and converted to lowercase. To retain source file parameter name capitalization, set spectral.settings.envi_support_nonlowercase_params to True.
  warnings.warn(msg)


Predicting: 0it [00:00, ?it/s]

...

Predicting DataLoader 0: 100%|██████████| 1/1 [00:01<00:00,  1.65s/it]
Threshold 0.280:
      Pixel Acc: 0.845
      DICE     : 0.196
      +IOU     : 0.109
      Avg Prec : 0.292

      Conf Mat : [0.8485277891159058, 0.15147224068641663]
                 [0.26990392804145813, 0.7300960421562195]
   Model: CubeNET_64
   Test JSON: /blue/azare/chang.spencer/HyperPRI/Datasets/HyperPRI/data_splits/test.json
   LOADING FROM CKPT FILE: /blue/azare/chang.spencer/HyperPRI/Saved_Models/HSI/CubeNET_64/Run_5/Checkpoints/epoch=98-val_loss=0.069-val_dice=0.807.ckpt


Exception ignored in: <function _ConnectionBase.__del__ at 0x1503d78013f0>
Traceback (most recent call last):
  File "/lustre/blue/azare/chang.spencer/textures/lib/python3.10/multiprocessing/connection.py", line 132, in __del__
    self._close()
  File "/lustre/blue/azare/chang.spencer/textures/lib/python3.10/multiprocessing/connection.py", line 361, in _close
    _close(self._handle)
OSError: [Errno 9] Bad file descriptor
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
SLURM auto-requeueing enabled. Setting signal handlers.
/lustre/blue/azare/chang.spencer/textures/lib/python3.10/site-packages/spectral/io/envi.py:175: UserWarning: Parameters with non-lowercase names encountered and converted to lowercase. To retain source file parameter name capitalization, set spectral.settings.envi_support_nonlowercase_params to True.
  warnings.warn(msg)


Predicting: 0it [00:00, ?it/s]

/lustre/blue/azare/chang.spencer/textures/lib/python3.10/site-packages/spectral/io/envi.py:175: UserWarning: Parameters with non-lowercase names encountered and converted to lowercase. To retain source file parameter name capitalization, set spectral.settings.envi_support_nonlowercase_params to True.
  warnings.warn(msg)


Predicting DataLoader 0: 100%|██████████| 1/1 [00:00<00:00,  8.98it/s]
Threshold 0.270:
      Pixel Acc: 0.974
      DICE     : 0.597
      +IOU     : 0.425
      Avg Prec : 0.687

      Conf Mat : [0.9795664548873901, 0.020433560013771057]
                 [0.24736806750297546, 0.7526319026947021]


In [4]:
# temp_dataset = exp_params.get_test_data()

In [11]:
# import json
# with open(exp_params.json_dir['test'], 'r') as f:
#     data_dict = json.load(f)

In [18]:
# box_names = list(data_dict.keys())
# box = box_names[-1]

# box_class = data_dict[box]['plant_folder']
# box_sz = data_dict[box]['resolution']
# img_dir = f"{temp_dataset.root}/{box_class}_{box_sz}/{data_dict['img_dir']}/"
# label_dir = f"{temp_dataset.root}/{box_class}_{box_sz}/{data_dict['mask_dir']}/"

In [20]:
# data_dict[box]['dates']

['20220815', '20220824']

## Manual Model Evaluation
For each model, evaluate the CPU inference time when using each model in evaluation mode. This is averaged across 5 runs.

In [10]:
spectral.settings.envi_support_nonlowercase_params = True
with torch.no_grad():

    # Number of times to average the inference time on GPU
    num_splits = 5
    
    for m_idx, (m, dset) in enumerate(zip(models, datasets)):
        print(f"Model: {m}")
        final_time = 0
        for run in range(start_split, num_splits):
            
            change_params = {}
            for k_idx, k in enumerate(update_params):
                change_params[k] = update_params[k][m_idx]

            # change_params['device'] = 'cpu'

            split_no = run + 1
            if dset.lower() == 'rgb':
                exp_params = ExpRedGreenBluePRI(rel_call_path, split_no=run+1, augment=False)
                # Switch to a different model (ie. change internal parameter strings)
                exp_params.change_network_param(m, rel_call_path, run+1, model_params=change_params)  # Num bins
            else:
                exp_params = ExpHyperspectralPRI(rel_call_path, split_no=run+1)
                # Switch to a different model (ie. change internal parameter strings)
                exp_params.change_network_param(m, rel_call_path, run+1, model_params=None)  # Num bins
            exp_params.json_dir['test'] = os.path.join(f"{exp_params.data_dir}", "data_splits", "test.json")

            net = exp_params.get_network()
            test_loader = DataLoader(exp_params.get_test_data(), batch_size=1)
            for batch in test_loader:
                X_img = batch['image']
                break
            
            if exp_params.device.lower() == 'gpu' and torch.cuda.is_available() and torch.cuda.device_count() > 0:
                net = net.to('cuda:0')
                X_img = X_img.to('cuda:0')

            t_start = time.time()
            pred = net(X_img)
            t_stop = time.time()

            del net
            del X_img

            time_diff = t_stop - t_start

            print(f"   {time_diff:.3f} s")
            final_time += time_diff
        print(f"   Average: {final_time / num_splits:.4f} s", end="\n\n")


Model: UNET
   0.018 s
   0.003 s
   0.003 s
   0.008 s
   0.003 s
   Average: 0.0072 s

Model: SpectralUNET
   0.017 s
   0.062 s
   0.002 s
   0.005 s
   0.002 s
   Average: 0.0174 s

Model: CubeNET
   0.011 s
   0.010 s
   0.007 s
   0.008 s
   0.045 s
   Average: 0.0159 s

